# Oppg 8b, Konfidensintervaller

In [1]:
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [4]:
portCem = pd.read_csv('PortlandCementData.csv', sep=';')
portCem.columns = ['Modified', 'Strength']
portCem_melt = pd.melt(portCem.reset_index(), id_vars=['index'], value_vars=['Modified', 'Strength'])
print(portCem_melt)

    index  variable  value
0       0  Modified  16.85
1       1  Modified  16.40
2       2  Modified  17.21
3       3  Modified  16.35
4       4  Modified  16.52
5       5  Modified  17.04
6       6  Modified  16.96
7       7  Modified  17.15
8       8  Modified  16.59
9       9  Modified  16.57
10      0  Strength  16.62
11      1  Strength  16.75
12      2  Strength  17.37
13      3  Strength  17.12
14      4  Strength  16.98
15      5  Strength  16.87
16      6  Strength  17.34
17      7  Strength  17.02
18      8  Strength  17.08
19      9  Strength  17.27


In [3]:
model = ols('value ~ C(variable)', data=portCem_melt).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

,sum_sq,df,F,PR(>F)
C(variable),0.38642,1.0,4.782426,0.042197
Residual,1.45440,18.0,NaN,NaN


In [4]:
SSe = anova_table['sum_sq'][1]
dfe = anova_table['df'][1]
alpha = 0.05
high_tail = stats.chi2.ppf(0.025, dfe)
low_tail = stats.chi2.ppf(0.975, dfe)
lower = SSe/low_tail
higher = SSe/high_tail
interval = [lower, higher]
print(interval)

[0.04613279646915774, 0.17670329828983342]


# Oppg 9

In [1]:
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import researchpy as rp

In [2]:
sires = pd.read_csv('SiresData.csv', sep=';')

In [3]:
sires.head()

,Milk,Sire
0,6330,3
1,6337,2
2,6613,5
3,6250,4
4,6363,1


In [7]:
model2 = ols('Milk ~ C(Sire)', data=sires).fit()
anova_table2 = sm.stats.anova_lm(model2, typ=1)
print(anova_table2)

            df      sum_sq        mean_sq         F    PR(>F)
C(Sire)    4.0  1251009.35  312752.337500  2.805913  0.040379
Residual  35.0  3901165.75  111461.878571       NaN       NaN


In [8]:
mean = sires['Milk'].mean()
sigma_2 = anova_table2['sum_sq'][1]/anova_table2['df'][1]

In [9]:
print(f'Mean: {mean}\nSigma_2: {round(sigma_2, 2)}')

Mean: 6518.15
Sigma_2: 111461.88


In [10]:
mean_col = sires.groupby(['Sire'])['Milk'].mean()
col_sires = sires.groupby(['Sire'])['Milk']
col_summary = rp.summary_cont(col_sires)
print(mean_col, col_summary)



Sire
1    6689.250
2    6282.000
3    6325.750
4    6671.125
5    6622.625
Name: Milk, dtype: float64       N      Mean        SD        SE  95% Conf.   Interval
Sire                                                       
1     8  6689.250  468.8977  165.7804  6297.2417  7081.2583
2     8  6282.000  150.5770   53.2370  6156.1145  6407.8855
3     8  6325.750  286.1152  101.1570  6086.5517  6564.9483
4     8  6671.125  436.9683  154.4916  6305.8104  7036.4396
5     8  6622.625  204.8602   72.4290  6451.3576  6793.8924


In [11]:
for i in range(1,6):
    print(f'Tau{i}: {round(mean_col[i]-mean,3)}')

Tau1: 171.1
Tau2: -236.15
Tau3: -192.4
Tau4: 152.975
Tau5: 104.475


### 95% konfidens intervall for MSe

In [12]:
anova_table2

,df,sum_sq,mean_sq,F,PR(>F)
C(Sire),4.0,1251009.35,312752.337500,2.805913,0.040379
Residual,35.0,3901165.75,111461.878571,NaN,NaN


In [13]:
sse = anova_table2['sum_sq'][1]
dfe = anova_table2['df'][1]
high_tail = stats.chi2.ppf(0.025, dfe)
low_tail = stats.chi2.ppf(0.975, dfe)
higher = sse/high_tail
lower = sse/low_tail
interval = [lower, higher]
print(f'The intervall for the error variance is:\n{lower, higher}')

The intervall for the error variance is:
(73325.56797465836, 189658.9196664783)


### 1 claims that avg milk production is over 150 liter higher than the total average, is this true? (9e)
### H0: tau1 <= 150, Ha: tau1 > 150

In [19]:
model2 = ols('Milk ~ C(Sire, Sum)', data=sires).fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                   Milk   R-squared:                       0.243
Model:                            OLS   Adj. R-squared:                  0.156
Method:                 Least Squares   F-statistic:                     2.806
Date:                Fri, 14 Aug 2020   Prob (F-statistic):             0.0404
Time:                        13:59:57   Log-Likelihood:                -286.52
No. Observations:                  40   AIC:                             583.0
Df Residuals:                      35   BIC:                             591.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept          6518.1500     52.78

#### Leser av tabellen og bruker verdier for å beregne T0 

In [49]:
tau_aprox = mean_col[1] - mean 
T0 = (tau_aprox - 150)/(105.576)
dfe = anova_table2['df'][1]

In [50]:
t_alpha_dfe = stats.t.ppf(0.95, dfe)

In [51]:
print(f'T0: {T0}, T_alpha: {t_alpha_dfe}')
T0 > t_alpha_dfe

T0: 0.19985602788512888, T_alpha: 1.6895724539637709


False

##### Kan ikke forkaste H0

## Oppg 9f) sire 4 sier at melker mer enn 6500 i gjennomsnitt

### H0: my4 <= 6500, Ha:  my4 > 6500

In [40]:
mean = sires['Milk'].mean()
mean_col = sires.groupby(['Sire'])['Milk'].mean()
sire4_mean = mean_col[4]
anova_table2

,df,sum_sq,mean_sq,F,PR(>F)
C(Sire),4.0,1251009.35,312752.337500,2.805913,0.040379
Residual,35.0,3901165.75,111461.878571,NaN,NaN


In [53]:
se_mean4 = (anova_table2['mean_sq'][1]/8)**0.5
T0 = (sire4_mean-6500) / se_mean4
df = anova_table2['df'][1]
t_alpha_dfe = stats.t.ppf(0.95, dfe)
print(f'T0: {T0}, T_alpha: {t_alpha_dfe}')
T0 > t_alpha_dfe

T0: 1.449757203931578, T_alpha: 1.6895724539637709


False

### Oppg 9g), max mean average annual milk on a 5% significance level 
##### Altså regn ut nedre del av CI

In [58]:
t_alpha_dfe = stats.t.ppf(0.95, dfe) #dfe is 35 (df of mean square error)
max_mean_claim = sire4_mean - se_mean4*t_alpha_dfe
print('The max amount to claim is the average,' 
      f'with a 5% significant level is:\n{max_mean_claim}')

The max amount to claim is the average,with a 5% significant level is:
6471.692925856503


### Oppg 9h), is the average of my4 ad my3 different?
H0: my4 = my3, Ha: my4 != my3

In [69]:
my4 = sire4_mean # fra tidligere oppgave
my3 = mean_col[3]
diff = my4-my3
se_m4_m3 = (2*anova_table2['mean_sq'][1]/8)**0.5
T_0 = diff/se_m4_m3
t_sig_level5 = stats.t.ppf(0.975, 35)
t_sig_level1 = stats.t.ppf(0.995, 35)
print(f'T0: {T_0}\n'
    f'Can you reject H0 at alpha=5%?: {T_0 > t_sig_level5} because alpha gives {round(t_sig_level5, 2)}\n'
        f'Can you reject H0 at alpha=1%?: {T_0 > t_sig_level1} because alpha gives {round(t_sig_level1, 2)}')


T0: 2.068986773833709
Can you reject H0 at alpha=5%?: True because alpha gives 2.03
Can you reject H0 at alpha=1%?: False because alpha gives 2.72
